Crear diferentes Notebooks para cada proceso realizado.

EDA (dimensiones y tipos de columnas del df, número de NA por columnas y por filas (sacar la proporción de ambos casos), histogramas de cada una de las variables, correlaciones de las diferentes variables, correlaciones de cada grupo por separado). Sacar otras ideas de otros Notebooks de EDA de Kaggle.

Diferentes métodos simples (Simple Imputer, KNN Imputer, Iterative Imputer). Fijarme en los Notebooks subidos en Kaggle para utilizar alguno para explicar mejor la parte más teórica.

LigthGBM (Toda la parte de código usada de ARShaik). Conclusiones sacadas de que solo merece la pena entrenar con las columnas F4. El resto no tiene apenas cambio de utilizar otros metodos mas simples, tampoco influye para mejorar usar el resto de columnas (F1, F2 & F3) para predecir las columnas F4.

NN (Toda la parte de código usada de C4rl05/V). Seguimos con la suposición de que es mejor centrarse en mejorar únicamente el performance de las columnas F4. Probar las diferentes suposiciones entrenando los modelos para cada columna con todo el dataset cambiando los NA por valores medios en dicha columna. Probar a entrenar la NN con los datos sin NA. Probar la parte de ir calculando el modelo para cada columna y en función del número de NA de cada fila. Idea de probar a entrenar el modelo con los datos sin NA, pero a la hora de realizar las predicciones como en las filas con varios NA vamos a tener que poner el valor medio ir reentrenando estos modelos sucesivamente con las nuevas predicciones para los otros valores.

In [7]:
## Import libraries

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#import tqdm
import datetime
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [4]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

import random
import math

CPU times: total: 2.45 s
Wall time: 6.05 s


In [77]:
os.getcwd()

'C:\\Users\\guill\\Documents\\Kaggle\\Tabular_Playground_Series_Jun_2022'

In [78]:
## Read data csv

data = pd.read_csv('data.csv').drop("row_id", axis=1)
data

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,...,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733
999996,-0.769106,-0.387363,-1.227469,0.601183,0.351161,0.219475,-0.530277,0.853452,0.608646,1.648023,...,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807
999997,0.147534,-0.715276,-0.465049,-1.988941,-1.594535,-1.044882,3.159455,-0.634108,0.890382,-1.212444,...,2.891854,3.105002,-3.470520,NaN,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502
999998,-1.709886,-0.813785,-1.866536,-0.179420,2.231478,1.460122,-0.220585,-0.118496,-0.140064,1.362596,...,-1.135003,-5.127360,-3.903728,-1.597023,0.893159,0.379434,0.846266,-1.085554,3.122423,0.004831


In [10]:
## Read submission csv

submission = pd.read_csv('sample_submission.csv')
submission

,row-col,value
0,0-F_1_14,0.0
1,0-F_3_23,0.0
2,1-F_3_24,0.0
3,2-F_1_2,0.0
4,2-F_4_2,0.0
...,...,...
999995,999993-F_4_2,0.0
999996,999994-F_3_10,0.0
999997,999994-F_4_9,0.0
999998,999997-F_3_14,0.0


In [5]:
## Read train and test data previously saved

train = pd.read_csv('train.csv')
train_na = pd.read_csv('train_na.csv')
test = pd.read_csv('test.csv')
test_na = pd.read_csv('test_na.csv')

In [12]:
## Split data into 4 df for each group of columns (1, 2, 3 & 4)

features = list(data.columns)
features_1, features_2, features_3, features_4 = [], [], [], []

F_col = [[], [], [], [], []]
for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F_col[i].append(feature)

## Original data

data_col = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    data_col[i] = data[F_col[i]]

In [11]:
## Split data into 4 df for each group of columns (1, 2, 3 & 4)

features = list(data.columns)
features_1, features_2, features_3, features_4 = [], [], [], []

F_col = [[], [], [], [], []]
for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F_col[i].append(feature)

## Original data

data_col = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    data_col[i] = data[F_col[i]]

## Train data

train_col = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    train_col[i] = train[F_col[i]]

## Train with NA data

train_col_na = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    train_col_na[i] = train_na[F_col[i]]

## Test data

test_col = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    test_col[i] = test[F_col[i]]

## Test with NA data

test_col_na = [[], [], [], [], []]
for i in [1, 2, 3, 4]:
    test_col_na[i] = test_na[F_col[i]]

NameError: name 'train' is not defined

In [13]:
%%time
# Defining model parameters...
BATCH_SIZE = 2048
EPOCHS     = 32 # For Testing Purposes, Increase for Final Submission...
VERBOSE    = 0 
NUM_FOLDS  = 3  # For Testing Purposes, Increase for Final Submission...

CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time
def nn_model():
    
    '''
    Function to define the Neuronal Network architecture...
    '''
    
    L2 = 65e-6
    activation_func = 'swish'
    inputs = Input(shape = (len(features) - 1)) # Remove current feature that's imputed
    
    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(L2), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    
    x = Dense(64, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(L2), 
          activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    
    x = Dense(32, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(L2), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(L2),
              activation = 'linear')(x)
    
    model = Model(inputs, x)
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
# Baseline Network = [64,32,16,1] With 16 Epochs...  *
# Test Two Network = [128,64,32,1] With 16 Epochs... **
# Test Three Network = [128,64,32,1] With 32 Epochs... ???

In [16]:
 %%time
# Defining model training function...
def fit_model(X_train, y_train, X_val, y_val, X_test, run = 0):
    '''
    '''
    lr_start = 0.01
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 4, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 12, verbose = 1, mode = 'min', restore_best_weights = True)
    callbacks = [lr, es]
    
    model = nn_model()
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.MeanSquaredError()
    
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    #print(f'Training Loss:{history_list[-1]["loss"][-1]:.5f}')
    callbacks, es, lr, history = None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    score = mean_absolute_error(y_val, y_val_pred)
    
    #print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
    #      f'| MSE: {score:.5f}')
    
    score_list.append(score)
    
    tst_data_scaled = scaler.transform(X_test)
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
## Data for NA per rows = 1

data_train = test_col[4].copy()
data_test = train_col_na[4].copy()

In [ ]:
%%time
## NN for NA per rows = 1

# Create empty lists to store NN training metrics and predictions


# Creates a loop, train in all -1 columns use -1 column as a target...
features = data_test.columns.to_list()
prediction_nn_1 = pd.DataFrame()

models_nn = dict()

for feat in features:
    print('Processing Colunm Name : ', feat)
    # Create empty lists to store NN training metrics and predictions
    history_list = []
    score_list   = []
    predictions  = []
    
    if data_test[feat].isnull().any():
        #print('Training Model For: ',feat)
        
        # Identify missing values...
        missing_values = list(np.where(data_test[feat].isnull())[0])

        trn_data = data_train
        tst_data = data_test.iloc[missing_values,]
        
        # Define kfolds for training purposes...
        kf = KFold(n_splits = NUM_FOLDS)

        for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_data)):
            #print(f' Training fold: {fold}...')
            X_train, X_val = trn_data.iloc[trn_idx].drop([feat],axis = 1), trn_data.iloc[val_idx].drop([feat], axis = 1)
            y_train, y_val = trn_data.iloc[trn_idx][feat], trn_data.iloc[val_idx][feat]
            X_test = tst_data.drop([feat], axis = 1)
            
            X_train, X_val = X_train.fillna(X_train.mean()), X_val.fillna(X_val.mean())
            X_test = X_test.fillna(X_test.mean())
            
            fit_model(X_train, y_train, X_val, y_val, X_test)
        
        mean_values = np.array(predictions).mean(axis = 0)
        imputed_data = data_test[feat]
        imputed_data.iloc[missing_values] = mean_values.ravel()
        prediction_nn_1 = pd.concat([prediction_nn_1, imputed_data],axis = 1)
    
    else:
        prediction_nn_1 = pd.concat([prediction_nn_1, data_test[feat]],axis = 1)    

Processing Colunm Name :  F_4_0
Processing Colunm Name :  F_4_1
Processing Colunm Name :  F_4_2
Processing Colunm Name :  F_4_3
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Processing Colunm Name :  F_4_4
Processing Colunm Name :  F_4_5
Processing Colunm Name :  F_4_6
Processing Colunm Name :  F_4_7
Processing Colunm Name :  F_4_8
Processing Colunm Name :  F_4_9
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Processing Colunm Name :  F_4_10
Processing Colunm Name :  F_4_11
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Processing Colunm Name :  F_4_12
Processing Colunm Name :  F_4_13
Processing Colunm Name :  F_4_14
CPU times: total: 33min 48s
Wall time: 11min 15s


In [27]:
data_col[4].isna().sum().sum()

272528

In [33]:
data_col_no_na = data_col[4][(data_col[4].isna().sum(axis=1) == 0)]

In [43]:
## Data for NA per rows > 1

data_train = data_col[4].copy()
data_test = data_col[4].copy()

In [44]:
%%time
## NN for NA per row > 1

# Create empty lists to store NN training metrics and predictions

# Creates a loop, train in all -1 columns use -1 column as a target...
features = data_test.columns.to_list()
prediction_nn_1 = pd.DataFrame()

models_nn = dict()

for feat in features:
    print('Processing Colunm Name : ', feat)
    # Create empty lists to store NN training metrics and predictions
    history_list = []
    score_list   = []
    predictions  = []
    
    if data_test[feat].isnull().any():
        #print('Training Model For: ',feat)
        
        # Identify missing values...
        not_missing_values_train = list(np.where(data_train[feat].isnull() == False)[0])
        missing_values_test = list(np.where(data_test[feat].isnull())[0])

        trn_data = data_train.iloc[not_missing_values_train,]
        tst_data = data_test.iloc[missing_values_test,]
        
        # Define kfolds for training purposes...
        kf = KFold(n_splits = NUM_FOLDS)

        for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_data)):
            #print(f' Training fold: {fold}...')
            X_train, X_val = trn_data.iloc[trn_idx].drop([feat],axis = 1), trn_data.iloc[val_idx].drop([feat], axis = 1)
            y_train, y_val = trn_data.iloc[trn_idx][feat], trn_data.iloc[val_idx][feat]
            X_test = tst_data.drop([feat], axis = 1)
            
            X_train, X_val = X_train.fillna(X_train.mean()), X_val.fillna(X_val.mean())
            X_test = X_test.fillna(X_test.mean())
            
            models_nn[feat] = fit_model(X_train, y_train, X_val, y_val, X_test)
        
        mean_values = np.array(predictions).mean(axis = 0)
        imputed_data = data_test[feat]
        imputed_data.iloc[missing_values_test] = mean_values.ravel()
        prediction_nn_1 = pd.concat([prediction_nn_1, imputed_data],axis = 1)
    
    else:
        prediction_nn_1 = pd.concat([prediction_nn_1, data_test[feat]],axis = 1)    

Processing Colunm Name :  F_4_0
567/567 [==============================] - 0s 613us/step
Processing Colunm Name :  F_4_1
568/568 [==============================] - 0s 657us/step
Processing Colunm Name :  F_4_2
578/578 [==============================] - 0s 602us/step
Processing Colunm Name :  F_4_3
564/564 [==============================] - 0s 624us/step
Processing Colunm Name :  F_4_4
562/562 [==============================] - 0s 628us/step
Processing Colunm Name :  F_4_5
565/565 [==============================] - 0s 645us/step
Processing Colunm Name :  F_4_6
573/573 [==============================] - 0s 637us/step
Processing Colunm Name :  F_4_7
563/563 [==============================] - 0s 623us/step
Processing Colunm Name :  F_4_8
568/568 [==============================] - 0s 695us/step
Processing Colunm Name :  F_4_9
571/571 [==============================] - 0s 638us/step
Processing Colunm Name :  F_4_10
570/570 [==============================] - 0s 604us/step
Processing Colunm Na

In [59]:
%%time

## LGBM Regressor

for i in [4]:
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    test_predictions_1 = pd.DataFrame()
    test_predictions_2 = pd.DataFrame()
    test_predictions_3 = pd.DataFrame()
    test_predictions_4 = pd.DataFrame()
    test_predictions_5 = pd.DataFrame()
    col_test_train = pd.DataFrame()
    col_test_test = pd.DataFrame()
    train_df = data_col[i].copy()
    test_df = data_col[i].copy()
    test_predictions_1 = data_col[i].copy()
    test_predictions_2 = data_col[i].copy()
    test_predictions_3 = data_col[i].copy()
    test_predictions_4 = data_col[i].copy()
    test_predictions_5 = data_col[i].copy()

    models_lgbm = dict()

    for column in data_col[i].columns:
        print('Processing Colunm Name : ', column)
        col_nan_ix_test = test_df[test_df[column].isnull()].index  # identify the rows which has NaN in column F_4_0
        col_nan_ix_train = train_df[train_df[column].isnull()].index  # identify the rows which has NaN in column F_4_0

        col_test_train = train_df.drop(col_nan_ix_train, axis = 0) 
        col_test_test = test_df[test_df.index.isin(col_nan_ix_test)] 

        X_test = col_test_train.drop([column],axis=1)
        y_test = col_test_train[column]

        models_lgbm[column] = LGBMRegressor(n_estimators = 10000)
        models_lgbm[column].fit(X_test,y_test)

        rmse_test = mean_squared_error(y_test, models_lgbm[column].predict(X_test), squared = False)
        print('RMSE for Test of this column : ', rmse_test)

        test_predictions_1[column][col_nan_ix_test] = models_lgbm[column].predict(col_test_test.drop([column],axis=1))
        

Processing Colunm Name :  F_4_0
RMSE for Test of this column :  0.6656534696357426
Processing Colunm Name :  F_4_1
RMSE for Test of this column :  0.4763748400744299
Processing Colunm Name :  F_4_2
RMSE for Test of this column :  0.14762706014720212
Processing Colunm Name :  F_4_3
RMSE for Test of this column :  0.26124079795993815
Processing Colunm Name :  F_4_4
RMSE for Test of this column :  0.3314777939304229
Processing Colunm Name :  F_4_5
RMSE for Test of this column :  0.3686191190617441
Processing Colunm Name :  F_4_6
RMSE for Test of this column :  0.7560190890448347
Processing Colunm Name :  F_4_7
RMSE for Test of this column :  0.48834705552653807
Processing Colunm Name :  F_4_8
RMSE for Test of this column :  0.09493132543460063
Processing Colunm Name :  F_4_9
RMSE for Test of this column :  0.2578091199760006
Processing Colunm Name :  F_4_10
RMSE for Test of this column :  0.13213389090259983
Processing Colunm Name :  F_4_11
RMSE for Test of this column :  0.32986814980446

In [54]:
%%time
## NN for NA per row > 1

# Create empty lists to store NN training metrics and predictions

data_test = data_col[4].copy()

# Creates a loop, train in all -1 columns use -1 column as a target...
features = data_test.columns.to_list()

#prediction_nn_2 = pd.DataFrame()
prediction_nn_3 = pd.DataFrame()
#prediction_nn_4 = pd.DataFrame()
#prediction_nn_5 = pd.DataFrame()

for feat in features:
    print('Processing Colunm Name : ', feat)
    # Create empty lists to store NN training metrics and predictions
    history_list = []
    score_list   = []
    predictions  = []
    
    if data_test[feat].isnull().any():
        #print('Training Model For: ',feat)
        
        # Identify missing values...
        missing_values_test = list(np.where(data_test[feat].isnull())[0])
        not_missing_values_train = list(np.where(data_train[feat].isnull() == False)[0])

        trn_data = data_train.iloc[not_missing_values_train,]
        tst_data = prediction_nn_1.iloc[missing_values_test,]

        X_train = trn_data.drop([feat], axis = 1)
        X_test = tst_data.drop([feat], axis = 1)

        model = models_nn[feat]
        
        scaler = StandardScaler()
        scaler.fit(X_train)

        tst_data_scaled = scaler.transform(X_test)
        tst_pred = model.predict(tst_data_scaled)
        predictions.append(tst_pred)
        
        mean_values = np.array(predictions).mean(axis = 0)
        imputed_data = prediction_nn_1[feat]
        imputed_data.iloc[missing_values_test] = mean_values.ravel()
        prediction_nn_3 = pd.concat([prediction_nn_3, imputed_data],axis = 1)
    
    else:
        prediction_nn_3 = pd.concat([prediction_nn_3, prediction_nn_1[feat]],axis = 1)    

Processing Colunm Name :  F_4_0
567/567 [==============================] - 0s 629us/step
Processing Colunm Name :  F_4_1
568/568 [==============================] - 0s 680us/step
Processing Colunm Name :  F_4_2
578/578 [==============================] - 0s 642us/step
Processing Colunm Name :  F_4_3
564/564 [==============================] - 0s 632us/step
Processing Colunm Name :  F_4_4
562/562 [==============================] - 0s 633us/step
Processing Colunm Name :  F_4_5
565/565 [==============================] - 0s 608us/step
Processing Colunm Name :  F_4_6
573/573 [==============================] - 0s 617us/step
Processing Colunm Name :  F_4_7
563/563 [==============================] - 0s 621us/step
Processing Colunm Name :  F_4_8
568/568 [==============================] - 0s 689us/step
Processing Colunm Name :  F_4_9
571/571 [==============================] - 0s 638us/step
Processing Colunm Name :  F_4_10
570/570 [==============================] - 0s 630us/step
Processing Colunm Na

In [61]:
## Create 5 new prediction using the previous prediction data

for i in range(1, 5):
    print('Predictions nº', i + 1)
    for column in data_col[4].columns:
        print('Processing Colunm Name : ', column)
        col_nan_ix_test = test_df[test_df[column].isnull()].index  # identify the rows which has NaN in column F_4_0
        locals()['test_predictions_{0}'.format(i+1)][column][col_nan_ix_test] = models_lgbm[column].predict(locals()['test_predictions_{0}'.format(i)][test_df.index.isin(col_nan_ix_test)].drop([column],axis=1))

Predictions nº 2
Processing Colunm Name :  F_4_0
Processing Colunm Name :  F_4_1
Processing Colunm Name :  F_4_2
Processing Colunm Name :  F_4_3
Processing Colunm Name :  F_4_4
Processing Colunm Name :  F_4_5
Processing Colunm Name :  F_4_6
Processing Colunm Name :  F_4_7
Processing Colunm Name :  F_4_8
Processing Colunm Name :  F_4_9
Processing Colunm Name :  F_4_10
Processing Colunm Name :  F_4_11
Processing Colunm Name :  F_4_12
Processing Colunm Name :  F_4_13
Processing Colunm Name :  F_4_14
Predictions nº 3
Processing Colunm Name :  F_4_0
Processing Colunm Name :  F_4_1
Processing Colunm Name :  F_4_2
Processing Colunm Name :  F_4_3
Processing Colunm Name :  F_4_4
Processing Colunm Name :  F_4_5
Processing Colunm Name :  F_4_6
Processing Colunm Name :  F_4_7
Processing Colunm Name :  F_4_8
Processing Colunm Name :  F_4_9
Processing Colunm Name :  F_4_10
Processing Colunm Name :  F_4_11
Processing Colunm Name :  F_4_12
Processing Colunm Name :  F_4_13
Processing Colunm Name :  F_4

In [13]:
## Simple Imputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
train_col_na[1][:] = imp.fit_transform(train_col_na[1])
train_col_na[3][:] = imp.fit_transform(train_col_na[3])

C:\Users\U351473\AppData\Local\Temp\ipykernel_106896\94324379.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_col_na[1][:] = imp.fit_transform(train_col_na[1])
C:\Users\U351473\AppData\Local\Temp\ipykernel_106896\94324379.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_col_na[1][:] = imp.fit_transform(train_col_na[1])
C:\Users\U351473\AppData\Local\Temp\ipykernel_106896\94324379.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [17]:
train_col_na[3].isna().sum().sum()

0

In [51]:
test_4_na_3

,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
1916,0.795409,-0.623789,-0.770260,-0.440950,0.215060,1.719414,1.441969,1.595714,-0.815760,-0.488741,-1.071421,4.214355,-4.727274,1.104052,-0.108112
2137,-3.389891,-2.247449,0.568458,-0.009249,4.401177,2.675807,2.983970,0.293305,0.015338,-0.594958,-0.262718,0.863743,1.335779,-1.277215,-0.444087
2215,-0.354968,-0.127725,0.092135,-0.325934,-0.849824,-0.967033,-2.641119,-0.740895,0.393292,-0.305581,0.128901,-1.094825,-3.912872,1.299774,0.277912
2396,1.981048,-3.418477,0.507936,-0.462010,0.065459,2.479947,-0.774990,2.533314,-1.117679,0.715366,0.636271,3.128637,1.333375,0.991761,-0.257106
3080,0.274585,-3.043946,-0.518985,0.330013,-0.180993,-1.654114,-1.916158,3.379293,0.176179,-0.541852,-0.339082,-2.961478,0.823927,1.599802,0.011015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995228,-0.049236,-2.852658,0.838019,0.554093,-4.211202,-4.276435,-0.805499,-1.343661,0.024478,0.853383,0.927234,-6.708188,4.327698,-0.526517,0.795305
996495,1.377608,2.395085,0.252500,0.027860,1.101640,3.327307,0.312895,4.060688,1.025454,-1.329940,0.201551,1.379496,-2.031798,1.698120,0.423992
997355,-0.979589,-1.151294,-1.952817,0.145374,-2.259924,0.436069,0.164168,2.822745,-0.028007,0.668826,-0.054950,0.928307,2.897985,-0.304072,0.074433
997382,2.202326,-1.605612,-0.401132,-0.159647,1.026073,-0.046044,2.773899,1.350840,-1.199721,0.695865,-0.080005,2.004351,-0.034295,0.398658,-0.973724


In [40]:
data_concat = pd.concat([train_col_na[1], train_col_na[2], train_col_na[3], data_completed], axis=1)

In [41]:
data_concat.isna().sum().sum()

0

In [40]:
#train_col_na[4][train_col_na[4].isna().sum(axis=1) == 1] = test_predictions_1[train_col_na[4].isna().sum(axis=1) == 1]
test_4_na_1 = prediction_nn_1[data_col[4].isna().sum(axis=1) == 1]

In [62]:
#train_col_na[4][train_col_na[4].isna().sum(axis=1) == 2] = test_predictions_3[train_col_na[4].isna().sum(axis=1) == 2]
test_4_na_2 = test_predictions_3[data_col[4].isna().sum(axis=1) == 2]

In [50]:
#train_col_na[4][train_col_na[4].isna().sum(axis=1) == 3] = test_predictions_1[train_col_na[4].isna().sum(axis=1) == 3]
test_4_na_3 = prediction_nn_2[data_col[4].isna().sum(axis=1) == 3]

In [55]:
#train_col_na[4][train_col_na[4].isna().sum(axis=1) == 4] = test_predictions_1[train_col_na[4].isna().sum(axis=1) == 4]
test_4_na_4 = prediction_nn_3[data_col[4].isna().sum(axis=1) == 4]

In [56]:
#train_col_na[4][train_col_na[4].isna().sum(axis=1) == 4] = test_predictions_1[train_col_na[4].isna().sum(axis=1) == 4]
test_4_na_5 = prediction_nn_3[data_col[4].isna().sum(axis=1) == 4]

In [15]:
test_4_1 = train_col[4][train_col_na[4].isna().sum(axis=1) == 1]

In [35]:
test_4_2 = train_col[4][train_col_na[4].isna().sum(axis=1) == 2]

In [22]:
test_4_3 = train_col[4][train_col_na[4].isna().sum(axis=1) == 3]

In [29]:
test_4_4 = train_col[4][train_col_na[4].isna().sum(axis=1) == 4]

In [22]:
test_4_na_1_2_3_4 = pd.concat([test_4_na_1, test_4_na_2, test_4_na_3, test_4_na_4],axis=0)

In [23]:
test_4_1_2_3_4 = pd.concat([test_4_1, test_4_2, test_4_3, test_4_4],axis=0)

In [62]:
## RMSE Cols DF Test (15000/25000) Train (60000/100000) 

error_df_br = prediction_nn_4.sub(train_col[4])
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 60000
np.sqrt(error)

1.2800208716692978

In [36]:
## RMSE Cols DF Test NA Count (11675, 2972, 333, 20 / 13277) Train NA Count (46674, 11836, 1374, 116 / 53079) 

error_df_br = test_4_na_2.sub(test_4_2)
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 11836
np.sqrt(error)

0.9607246801622356

In [55]:
data_concat = pd.concat([train_col_na[1], train_col_na[2], train_col_na[3], prediction_nn_3], axis=1)

In [56]:
## RMSE DF Test (55000) Train (220000)

error_df_br = data_concat.sub(train)
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 220000
np.sqrt(error)

1.0949812403095471

In [66]:
## Count of NA per row

na_cnt = pd.DataFrame(data_col[4].isna().sum(axis=1))

In [67]:
na_cnt.groupby([0]).size()

0
0     999996
15         4
dtype: int64

In [61]:
## Best Predictions

In [ ]:
## NA = 1 -> NN with No NA (1 iter)
## NA = 2 -> LGBM with NA (3)
## NA = 3 -> NN with NA (2)
## NA = 4 -> NN with NA (3)

In [14]:
#final_test_na_1 = test_4_na_1

In [37]:
#final_test_4_na_2 = test_4_na_2

In [24]:
#final_test_4_na_3 = test_4_na_3

In [41]:
#final_test_4_na_4 = test_4_na_4

In [65]:
data_col[4].isna().sum().sum()

60

In [63]:
data_col[4][data_col[4].isna().sum(axis=1) == 1] = test_4_na_1
data_col[4][data_col[4].isna().sum(axis=1) == 2] = test_4_na_2
data_col[4][data_col[4].isna().sum(axis=1) == 3] = test_4_na_3
data_col[4][data_col[4].isna().sum(axis=1) == 4] = test_4_na_4
data_col[4][data_col[4].isna().sum(axis=1) == 5] = test_4_na_5

C:\Users\guill\AppData\Local\Temp\ipykernel_29252\3842458909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 1] = test_4_na_1
C:\Users\guill\AppData\Local\Temp\ipykernel_29252\3842458909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 1] = test_4_na_1
C:\Users\guill\AppData\Local\Temp\ipykernel_29252\3842458909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [68]:
## Simple Imputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
data_col[1][:] = imp.fit_transform(data_col[1])
data_col[3][:] = imp.fit_transform(data_col[3])

C:\Users\guill\AppData\Local\Temp\ipykernel_29252\2674904411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[1][:] = imp.fit_transform(data_col[1])
C:\Users\guill\AppData\Local\Temp\ipykernel_29252\2674904411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[1][:] = imp.fit_transform(data_col[1])
C:\Users\guill\AppData\Local\Temp\ipykernel_29252\2674904411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [69]:
data_concat = pd.concat([data_col[1], data_col[2], data_col[3], data_col[4]], axis=1)

In [73]:
data_concat.isna().sum().sum()

0

In [47]:
## RMSE Cols DF Test (15000/25000) Train (60000/100000) 

error_df_br = train_col_na[4].sub(train_col[4])
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 60000
np.sqrt(error)

0.5412321992730276

In [ ]:
## RMSE Cols DF Test NA Count (11675, 2972, 333, 20 / 13277) Train NA Count (46674, 11836, 1374, 116 / 53079) 

error_df_br = test_4_na_2.sub(test_4_2)
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 11836
np.sqrt(error)

0.9607246801622356

In [48]:
## RMSE DF Test (55000) Train (220000)

error_df_br = data_concat.sub(train)
error_df_br_abs = error_df_br ** 2
error = error_df_br_abs.sum().sum() / 220000
np.sqrt(error)

0.8731908097815179

In [59]:
## Submission

In [74]:
#sub = automated_sub_func_melt(data_imputer, submission)
submission["value"] = submission["row-col"].apply(lambda item: data_concat.iloc[int(item.split("-")[0])][item.split("-")[1]])

In [75]:
submission

,row-col,value
0,0-F_1_14,-0.000905
1,0-F_3_23,0.000365
2,1-F_3_24,-0.000817
3,2-F_1_2,0.000551
4,2-F_4_2,0.322784
...,...,...
999995,999993-F_4_2,-0.114520
999996,999994-F_3_10,0.001706
999997,999994-F_4_9,-0.134452
999998,999997-F_3_14,0.000727


In [76]:
submission.to_csv('submission.csv', index=False) 

In [31]:
## Count of NA per row

na_cnt = pd.DataFrame(data_col[4].isna().sum(axis=1))

In [32]:
na_cnt.groupby([0]).size()

0
0    759268
1    211342
2     27127
3      2124
4       135
5         4
dtype: int64

In [30]:
data_col[4].isna().sum().sum()

272528

In [39]:
data_col[4][data_col[4].isna().sum(axis=1) == 5]

,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
225098,0.268807,NaN,-0.075833,0.274833,NaN,-4.322828,NaN,NaN,0.751367,0.726863,1.349478,0.701888,NaN,2.243863,1.179875
604135,1.473255,-2.091601,NaN,0.321455,2.944792,-0.845840,-0.976769,-1.096915,NaN,NaN,NaN,5.935633,NaN,2.229738,1.112853
857976,-1.640267,NaN,0.355139,NaN,-1.320024,-0.344264,NaN,2.175051,0.118577,NaN,-0.072447,NaN,-2.613657,-0.566197,-0.050125
886281,1.467103,NaN,0.823892,-0.273274,0.560576,NaN,-3.484972,0.720048,NaN,0.290900,-0.093264,NaN,-2.919703,-5.173954,NaN


In [40]:
data_col[4][data_col[4].isna().sum(axis=1) == 1] = data_completed_1[data_col[4].isna().sum(axis=1) == 1]

C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\4211179680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 1] = data_completed_1[data_col[4].isna().sum(axis=1) == 1]
C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\4211179680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 1] = data_completed_1[data_col[4].isna().sum(axis=1) == 1]


In [41]:
data_col[4][data_col[4].isna().sum(axis=1) == 2] = data_completed[data_col[4].isna().sum(axis=1) == 2]

C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\1994812681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 2] = data_completed[data_col[4].isna().sum(axis=1) == 2]
C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\1994812681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 2] = data_completed[data_col[4].isna().sum(axis=1) == 2]


In [42]:
data_col[4][data_col[4].isna().sum(axis=1) == 3] = data_completed[data_col[4].isna().sum(axis=1) == 3]

C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\674399112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 3] = data_completed[data_col[4].isna().sum(axis=1) == 3]
C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\674399112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 3] = data_completed[data_col[4].isna().sum(axis=1) == 3]


In [43]:
data_col[4][data_col[4].isna().sum(axis=1) == 4] = data_completed[data_col[4].isna().sum(axis=1) == 4]

C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\3030648619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 4] = data_completed[data_col[4].isna().sum(axis=1) == 4]
C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\3030648619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 4] = data_completed[data_col[4].isna().sum(axis=1) == 4]


In [44]:
data_col[4][data_col[4].isna().sum(axis=1) == 5] = data_completed[data_col[4].isna().sum(axis=1) == 5]

C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\2573207123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 5] = data_completed[data_col[4].isna().sum(axis=1) == 5]
C:\Users\U351473\AppData\Local\Temp\ipykernel_96836\2573207123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_col[4][data_col[4].isna().sum(axis=1) == 5] = data_completed[data_col[4].isna().sum(axis=1) == 5]


In [50]:
data_concat.isna().sum().sum()

0

In [49]:
data_concat = pd.concat([data_col[1], data_col[2], data_col[3], data_col[4]], axis = 1)

In [51]:
#sub = automated_sub_func_melt(data_imputer, submission)
submission["value"] = submission["row-col"].apply(lambda item: data_concat.iloc[int(item.split("-")[0])][item.split("-")[1]])

In [52]:
submission

,row-col,value
0,0-F_1_14,-0.000905
1,0-F_3_23,0.000365
2,1-F_3_24,-0.000817
3,2-F_1_2,0.000551
4,2-F_4_2,0.360833
...,...,...
999995,999993-F_4_2,-0.139782
999996,999994-F_3_10,0.001706
999997,999994-F_4_9,-0.134005
999998,999997-F_3_14,0.000727


In [53]:
submission.to_csv('submission.csv', index=False) 